In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from vamos import optimize
from vamos.algorithms import NSGAIIConfig
from vamos.problems import ZDT1
from vamos.problems import DTLZ2
from vamos.foundation.kernel.registry import KERNELS, resolve_kernel

plt.style.use("ggplot")


def make_autonsgaii_config():
    return (
        NSGAIIConfig.builder()
        .pop_size(56)
        .offspring_size(14)
        .crossover("blx_alpha", prob=0.88, alpha=0.94, repair="clip")
        .mutation("non_uniform", prob="0.45/n", perturbation=0.3)
        .selection("tournament", pressure=9)
        .repair("round")
        .external_archive(size=56, archive_type="hypervolume")
        
        .build()
    )


# Check available backends
print("Registered backends:", list(KERNELS.keys()))

# Test which ones are actually available
available = []
for name in KERNELS:
    try:
        resolve_kernel(name)
        available.append(name)
        print(f"  ✓ {name}: available")
    except ImportError as e:
        print(f"  ✗ {name}: {e}")

print(f"\nUsable backends: {available}")

## 1. Backend Selection

Select a backend via `optimize(..., engine=...)`:

In [ ]:
# NumPy backend (always available)
numpy_config = make_autonsgaii_config()

# Numba backend (if installed)
try:
    numba_config = make_autonsgaii_config()
    has_numba = True
    print("Numba backend: available")
except Exception:
    has_numba = False
    print("Numba backend: not installed")

# MooCore backend (if installed)
try:
    moocore_config = make_autonsgaii_config()
    has_moocore = True
    print("MooCore backend: available")
except Exception:
    has_moocore = False
    print("MooCore backend: not installed")

## 2. Performance Benchmark

In [ ]:
def benchmark_backend(engine_name, problem, max_evals=5000, seed=42):
    """Run optimization and measure time."""
    config = make_autonsgaii_config()

    start = time.time()
    result = optimize(
        problem=problem,
        algorithm="nsgaii",
        algorithm_config=config,
        termination=("n_eval", max_evals),
        seed=seed,
        engine=engine_name,
    )
    elapsed = time.time() - start

    return {
        "engine": engine_name,
        "time": elapsed,
        "solutions": len(result.F),
        "result": result,
    }

In [ ]:
# Benchmark available backends
zdt1 = ZDT1(n_var=30)
MAX_EVALS = 10000

benchmarks = {}

print(f"Benchmarking on ZDT1 ({MAX_EVALS} evaluations)...")
print("-" * 50)

# NumPy (always available)
benchmarks["numpy"] = benchmark_backend("numpy", zdt1, MAX_EVALS)
print(f"NumPy:   {benchmarks['numpy']['time']:.3f}s")

# Numba (if available)
if has_numba:
    # Warm-up run for JIT compilation
    _ = benchmark_backend("numba", zdt1, 500)
    benchmarks["numba"] = benchmark_backend("numba", zdt1, MAX_EVALS)
    print(f"Numba:   {benchmarks['numba']['time']:.3f}s")

# MooCore (if available)
if has_moocore:
    benchmarks["moocore"] = benchmark_backend("moocore", zdt1, MAX_EVALS)
    print(f"MooCore: {benchmarks['moocore']['time']:.3f}s")

In [ ]:
# Visualize benchmark results
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Runtime comparison
engines = list(benchmarks.keys())
times = [benchmarks[e]["time"] for e in engines]
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(engines)))

axes[0].bar(engines, times, color=colors)
axes[0].set_ylabel("Runtime (seconds)")
axes[0].set_title(f"Backend Runtime ({MAX_EVALS} evals)")
for i, (e, t) in enumerate(zip(engines, times)):
    axes[0].text(i, t + 0.05, f"{t:.2f}s", ha="center", fontsize=10)

# Speedup relative to NumPy
numpy_time = benchmarks["numpy"]["time"]
speedups = [numpy_time / benchmarks[e]["time"] for e in engines]

axes[1].bar(engines, speedups, color=colors)
axes[1].axhline(1.0, color="red", linestyle="--", alpha=0.5, label="NumPy baseline")
axes[1].set_ylabel("Speedup (vs NumPy)")
axes[1].set_title("Relative Performance")
for i, (e, s) in enumerate(zip(engines, speedups)):
    axes[1].text(i, s + 0.05, f"{s:.2f}x", ha="center", fontsize=10)
axes[1].legend()

plt.tight_layout()
plt.show()

## 3. Scaling with Population Size

In [ ]:
# Test scaling with population size
pop_sizes = [56]
scaling_results = {e: [] for e in available}

print("Testing population scaling...")
for pop_size in pop_sizes:
    print(f"  Pop size {pop_size}: ", end="")
    for engine in available:
        config = make_autonsgaii_config()
        start = time.time()
        _ = optimize(
            problem=zdt1,
            algorithm="nsgaii",
            algorithm_config=config,
            termination=("n_eval", 5000),
            seed=42,
            engine=engine,
        )
        elapsed = time.time() - start
        scaling_results[engine].append(elapsed)
        print(f"{engine}={elapsed:.2f}s ", end="")
    print()

In [ ]:
# Plot scaling results
fig, ax = plt.subplots(figsize=(8, 5))

markers = ["o", "s", "^"]
for i, engine in enumerate(available):
    ax.plot(pop_sizes, scaling_results[engine], marker=markers[i % len(markers)], linewidth=2, markersize=8, label=engine.upper())

ax.set_xlabel("Population Size")
ax.set_ylabel("Runtime (seconds)")
ax.set_title("Backend Scaling with Population Size")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Backend-Specific Features

### NumPy Backend
- Pure NumPy vectorized operations
- No additional dependencies
- Good baseline performance

In [ ]:
# NumPy kernel details
from vamos.foundation.kernel.numpy_backend import NumPyKernel

numpy_kernel = NumPyKernel()
print("NumPy Kernel Operations:")
print(f"  - fast_non_dominated_sort()")
print(f"  - compute_crowding()")
print(f"  - survival_selection()")
print(f"  - create_offspring()")

### Numba Backend
- JIT-compiled hot paths
- First run has compilation overhead
- Significant speedup on large populations

In [ ]:
if has_numba:
    print("Numba Backend Features:")
    print("  - JIT-compiled non-dominated sorting")
    print("  - JIT-compiled crowding distance")
    print("  - Cached compilation (faster on subsequent runs)")
    print("\n  Note: First run includes JIT compilation overhead.")
    print("        Subsequent runs are significantly faster.")
else:
    print("Numba not installed. Install with:")
    print("  pip install numba>=0.57")
    print("  # or")
    print('  pip install -e ".[backends]"')

In [ ]:
if has_numba:
    print("Numba JIT Warm-up Analysis:")
    print("---------------------------")

    # Use a fresh problem type (DTLZ2) to potentially trigger new JIT paths
    warmup_prob = DTLZ2(n_var=10, n_obj=3)
    warmup_config = make_autonsgaii_config()

    # Run 1: Cold Start (Compilation + Execution)
    start1 = time.time()
    _ = optimize(
        problem=warmup_prob, algorithm="nsgaii", algorithm_config=warmup_config, termination=("n_eval", 1000), seed=1
    )
    cold_time = time.time() - start1

    # Run 2: Warm Start (Execution only)
    start2 = time.time()
    _ = optimize(
        problem=warmup_prob, algorithm="nsgaii", algorithm_config=warmup_config, termination=("n_eval", 1000), seed=2
    )
    warm_time = time.time() - start2

    print(f"Cold Run (Compilation): {cold_time:.4f}s")
    print(f"Warm Run (Execution):   {warm_time:.4f}s")
    print(f"JIT Overhead:           {cold_time - warm_time:.4f}s")

### MooCore Backend
- C++ implementation of performance indicators
- Particularly fast for hypervolume calculations
- Required for SMS-EMOA efficiency

In [ ]:
if has_moocore:
    from vamos.foundation.metrics import moocore_indicators as mi

    print("MooCore Backend Features:")
    print("  - Fast hypervolume calculation")
    print("  - Epsilon indicator")
    print("  - IGD/IGD+ indicators")

    # Benchmark hypervolume
    F_test = np.random.rand(100, 2)
    ref = np.array([1.1, 1.1])

    start = time.time()
    for _ in range(100):
        hv = mi.hv_wrapper(F_test, ref)
    moocore_time = time.time() - start

    print(f"\n  MooCore HV: {moocore_time:.4f}s for 100 calculations")
else:
    print("MooCore not installed. Install with:")
    print("  pip install moocore>=0.4")
    print("  # or")
    print('  pip install -e ".[backends]"')

## 5. Solution Quality Verification

Verify that all backends produce equivalent results:

In [ ]:
# Compare solution quality across backends
from vamos.foundation.metrics.hypervolume import hypervolume

ref_point = np.array([1.1, 1.1])

print("Solution Quality Comparison:")
print("-" * 50)
print(f"{'Backend':<12} {'Solutions':>10} {'HV':>12}")
print("-" * 50)

for engine, data in benchmarks.items():
    hv = hypervolume(data["result"].F, ref_point)
    print(f"{engine:<12} {data['solutions']:>10} {hv:>12.6f}")

print("\nNote: Small HV differences are due to stochastic nature.")

In [ ]:
from vamos.foundation.metrics.pareto import pareto_filter

# Overlay Pareto fronts from different backends
fig, ax = plt.subplots(figsize=(9, 6))

fronts = []
for data in benchmarks.values():
    F = data["result"].front()
    if F is not None and F.size:
        fronts.append(F)

if fronts:
    F_all = np.vstack(fronts)
    F_plot = pareto_filter(F_all)
    if F_plot is not None and F_plot.size:
        ax.scatter(F_plot[:, 0], F_plot[:, 1], s=30, alpha=0.7, c="steelblue", label="Union front")

# True Pareto front
f1_true = np.linspace(0, 1, 100)
f2_true = 1 - np.sqrt(f1_true)
ax.plot(f1_true, f2_true, "k--", linewidth=2, alpha=0.5, label="True PF")

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title("Backend Comparison: Pareto Fronts")
ax.legend(loc="upper right")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Recommendations

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    BACKEND SELECTION GUIDE                           ║
╠══════════════════════════════════════════════════════════════════════╣
║ SCENARIO                      │ RECOMMENDED BACKEND                  ║
╠═══════════════════════════════╪══════════════════════════════════════╣
║ Quick experiments, portability│ NumPy (default, no dependencies)     ║
║ Large populations (>200)      │ Numba (JIT speedup)                  ║
║ Many evaluations (>50k)       │ Numba (accumulated speedup)          ║
║ SMS-EMOA algorithm            │ MooCore (fast HV calculation)        ║
║ Production/benchmarking       │ Numba or MooCore                     ║
╠═══════════════════════════════╧══════════════════════════════════════╣
║ INSTALL BACKENDS:                                                    ║
║   pip install -e ".[backends]"  # Installs numba + moocore           ║
╚══════════════════════════════════════════════════════════════════════╝
""")

## Summary

**Backend Selection:**
```python
# NumPy (default, always available)
config = make_autonsgaii_config()

# Numba (faster, requires numba package)
config = make_autonsgaii_config()

# MooCore (fastest for HV, requires moocore package)
config = make_autonsgaii_config()
```

**Installation:**
```bash
# Core only (NumPy backend)
pip install -e .

# With accelerated backends
pip install -e ".[backends]"
```

**Performance Tips:**
1. Use NumPy for quick experiments and prototyping
2. Switch to Numba for production runs with large populations
3. Use MooCore for hypervolume-intensive algorithms (SMS-EMOA)
4. First Numba run includes JIT compilation overhead
